<a href="https://colab.research.google.com/github/Davidxswang/ML/blob/master/Note_2_TensorFlow_Keras_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A easy demo provided by TensorFlow tutorial

## Import the packages

In [0]:
import tensorflow as tf

def println(*arg, **argm):
  print(*arg, **argm)
  print('\n')

In [2]:
# In note 1, we import the data from TensorFlow Datasets. In this notebook, MNIST dataset is imported from tensorflow.keras
mnist = tf.keras.datasets.mnist

# Let's see what type mnist is and what methods it has
println('Class of mnist: ', type(mnist))
println('Methods: ', dir(mnist))

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Let's take a look at the x and y data
println('Type of X:', type(x_train),'Type of Y:', type(y_train))
println('Shape of train data X and Y:', x_train.shape, y_train.shape)
println('Shape of test data X and Y:', x_test.shape, y_test.shape)

Class of mnist:  <class 'module'>


Methods:  ['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_sys', 'load_data']


11493376/11490434 [==============================] - 0s 0us/step
Type of X: <class 'numpy.ndarray'> Type of Y: <class 'numpy.ndarray'>


Shape of train data X and Y: (60000, 28, 28) (60000,)


Shape of test data X and Y: (10000, 28, 28) (10000,)




## Preprocess the data

In [0]:
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a simple model

In [4]:
# Using a sequential model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Take a look at the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Model can be used directly to make predictions
predictions = model(x_train[:5])
println('Type of model output:',type(predictions))
# Make the eager tensor a numpy array
predictions = predictions.numpy()
println('Shape of output:', predictions.shape)
# Predictions in logits
println('Predictions in logits:\n',predictions)

# We can convert it to softmax
softmax = tf.math.softmax(predictions)
println('Softmax scores:\n', softmax.numpy())

# To extract the final answer, use argmax
answers = tf.math.argmax(softmax)
println('answers:\n', answers.numpy())

Type of model output: <class 'tensorflow.python.framework.ops.EagerTensor'>


Shape of output: (5, 10)


Predictions in logits:
 [[-1.0871801   0.45626637 -0.20547834 -0.11279161 -0.41682982  0.82375824
  -0.99967855  0.54372233 -0.1992756   0.07739048]
 [-1.0812563   0.21659216 -0.5500584  -0.38242614 -0.27764216  0.6078838
  -0.676491    0.40565526 -0.05005355 -0.11835589]
 [-0.6775217  -0.36640644  0.5077245  -0.33400747 -0.20692378  0.19915411
  -0.19673057  0.32317096 -0.38205656 -0.17919885]
 [-0.44259685 -0.22398242  0.33039337 -0.31359267 -0.6854013   0.02962984
  -0.2934435   0.12106162 -1.1286916   0.22544752]
 [-0.9232533   0.7812769   0.5160189   0.5047922  -0.03776971  0.35309613
  -0.49114504  0.81394076 -0.04444627 -0.27916297]]


Softmax scores:
 [[0.0319549  0.14957112 0.07717128 0.08466599 0.06246934 0.21599686
  0.03487699 0.16324106 0.07765143 0.10240096]
 [0.03653865 0.13378298 0.06215109 0.07349377 0.08161268 0.1978503
  0.05476963 0.16162552 0.10247033 0.09570504

In [6]:
# We can calculate the loss by compare softmax with y, or by compare logits with y
# Logits with y
sparselogitloss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_1 = sparselogitloss(y_train[:5], predictions).numpy()
println('Loss calculated by logits and y:',loss_1)

# softmax with y
sparsesoftmaxloss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss_2 = sparsesoftmaxloss(y_train[:5], softmax).numpy()
println('Loss calculated by softmax and y:', loss_2)

# This type of loss should be negative log possibility
loss_3_guess = -tf.math.log(1/10).numpy()
println('If we guess, the loss should be:', loss_3_guess)
# So it's very close to guess randomly when the network is not trained at all

Loss calculated by logits and y: 2.4974241


Loss calculated by softmax and y: 2.4974241


If we guess, the loss should be: 2.3025851




In [18]:
# designate the optimizer, loss function and metrics
model.compile(optimizer='adam', loss=sparselogitloss, metrics='acc')

# train the network
# default batch_size is 32 (batch_size=None)
history = model.fit(x_train, y_train, epochs=5, batch_size=60000)

# let's take a look at history
println('Type of history:',history)
println('Method of history:', dir(history))
println('Commonly used history:', type(history.history),'keys of dict:',list(history.history.keys()))
println('Type of loss and acc:', type(history.history['loss']),type(history.history['acc']))
println('Type of epoch:', type(history.epoch))


Epoch 1/500
1/1 [==============================] - 0s 1ms/step - loss: 0.0084 - acc: 0.9978
Epoch 2/500
1/1 [==============================] - 0s 1ms/step - loss: 0.0084 - acc: 0.9978
Epoch 3/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0089 - acc: 0.9976
Epoch 4/500
1/1 [==============================] - 0s 1ms/step - loss: 0.0089 - acc: 0.9975
Epoch 5/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0086 - acc: 0.9979
Epoch 6/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0084 - acc: 0.9979
Epoch 7/500
1/1 [==============================] - 0s 969us/step - loss: 0.0086 - acc: 0.9977
Epoch 8/500
1/1 [==============================] - 0s 1ms/step - loss: 0.0090 - acc: 0.9974
Epoch 9/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0079 - acc: 0.9977
Epoch 10/500
1/1 [==============================] - 0s 976us/step - loss: 0.0083 - acc: 0.9980
Epoch 11/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0

In [20]:
evaluate = model.evaluate(x_test, y_test)
println('Type of evaluate:', type(evaluate), 'content:', evaluate)
loss, acc = evaluate
println(f'Loss is: {loss}, accuracy is: {acc}')

313/313 [==============================] - 0s 2ms/step - loss: 0.0937 - acc: 0.9801
Type of evaluate: <class 'list'> content: [0.0936652347445488, 0.9800999760627747]


Loss is: 0.0936652347445488, accuracy is: 0.9800999760627747




## A harder demo provided by TensorFlow tutorial

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [22]:
# We will continue using mnist dataset
# But as we can see the mnist dataset Keras provides doesn't have a channel dimension, we need to add one at the tail
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)
println(f'Shape of x_train is {x_train.shape}, shape of x_test is {x_test.shape}')

Shape of x_train is (60000, 28, 28, 1), shape of x_test is (10000, 28, 28, 1)


